In [1]:
from utils.util import load_from_txt,discard_overlaps,iou, write_to_csv_file
import cv2
from collections import defaultdict
import numpy as np
import imageio
import copy
import os

In [2]:
def frame_visualization(video_capture,predicted_boxes,frame_id,out_path,method):
    video_capture.set(cv2.CAP_PROP_POS_FRAMES, frame_id - 1)
    res, frame = video_capture.read()
    # Draw the ground truth boxes
    # Draw the predicted boxes
    for box in predicted_boxes[frame_id]:
        cv2.rectangle(frame, (int(box[1]), int(box[2])), (int(box[3]), int(box[4])), (0, 0, 255), 2)
        cv2.putText(frame, f'id:{box[6]}',(int(box[1]), int(box[2])), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255),2)
    # put text mIOU of frame
    # put text number of frame
    cv2.putText(frame, f"Frame: {frame_id}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imwrite(f'{out_path}/{method}_{frame_id}.png', frame)


In [3]:
current_path = os.getcwd()


In [4]:
mask_rcnn = '../../dataset/AICity_data/train/S03/c010/det/det_mask_rcnn.txt'
mask_rcnn = os.path.join(current_path,mask_rcnn)
ssd512 =  '../../dataset/AICity_data/train/S03/c010/det/det_ssd512.txt'
ssd512 = os.path.join(current_path,ssd512)
yolo3 =  '../../dataset/AICity_data/train/S03/c010/det/det_yolo3.txt'
yolo3 = os.path.join(current_path,yolo3)

In [5]:
video = '../../dataset/AICity_data/train/S03/c010/vdo.avi'
video = os.path.join(current_path,video)
video_capture = cv2.VideoCapture(video)


In [6]:
mask_rcnn_boxes = load_from_txt(mask_rcnn,threshold=0.5)

In [7]:
def track_memory(tracked_objects):
    delete = []
    for idx in tracked_objects:
        if tracked_objects[idx]['memory'] != tracked_objects[idx]['frame']:
            if tracked_objects[idx]['memory'] <= 5:
                delete.append(idx)
 

    for idx in delete:
        del tracked_objects[idx]

In [8]:
track_id = 0
tracked_objects = {}
memory = 5
for frame_id in mask_rcnn_boxes:
    # REMOVE OVERLAPPING BOUNDING BOXES
    boxes = mask_rcnn_boxes[frame_id]
    frame_boxes = discard_overlaps(boxes)
    # FIRST FRAME, WE INITIALIZE THE OBJECTS ID
    if frame_id == 0:
        for j in range(len(frame_boxes)):
            # We add the tracking object ID at the end of the list  [[frame,x1, y1, x2, y2, conf, track_id]]
            frame_boxes[j].append(track_id)
            tracked_objects[f'{track_id}'] = {'bbox':[frame_boxes[j][1],frame_boxes[j][2],frame_boxes[j][3],frame_boxes[j][4]],'frame':frame_id,'memory':0, 'iou':0}
            track_id += 1
            
    else:
        if frame_id == 150:
            break
        # FRAME N+1 WE COMPARE TO OBJECTS IN FRAME N
        for i in range(len(frame_boxes)):
            best_iou = 0
            track_id_best = 0
            boxA = [frame_boxes[i][1],frame_boxes[i][2],frame_boxes[i][3],frame_boxes[i][4]]

            for data in previous_tracked_objects.items():
                id,boxB = data
                iou_score,_ = iou(boxA,boxB['bbox'])

                if iou_score > best_iou and iou_score > 0.5:
                    best_iou = iou_score
                    track_id_best = id

            if track_id_best == 0 and best_iou == 0:
                frame_boxes[i].append(track_id)
                tracked_objects[f'{track_id}']  = {'bbox':boxA,'frame':frame_id,'memory':0,'iou':best_iou}
                track_id += 1
                
            else:
                if tracked_objects[f'{track_id_best}']['frame'] == frame_id:
                    # CHECK IF THERE IS AN OBJECT WITH THE SAME ID IN THAT FRAME AND CHOOSE THE ONE WITH HIGHEST IOU
                    if best_iou > tracked_objects[f'{track_id_best}']['iou']:
                        tracked_objects[f'{track_id}'] = {'bbox':tracked_objects[f'{track_id_best}']['bbox'],'frame':frame_id,'memory':0,'iou':best_iou}
                        wrong_id =  [i for i,det in enumerate(frame_boxes) if det[-1] == track_id_best][0]
                        frame_boxes[wrong_id][-1] = track_id
                        track_id += 1

                        frame_boxes[i].append(track_id_best)
                        tracked_objects[f'{track_id_best}']['bbox']= boxA
                        previous_f = tracked_objects[f'{track_id_best}']['frame']

                        # CHECK IF OBJECTS APPEAR CONSECUTIVE
                        if frame_id - previous_f == 1:
                            tracked_objects[f'{track_id_best}']['memory'] = tracked_objects[f'{track_id_best}']['memory'] + 1
                        tracked_objects[f'{track_id_best}']['frame'] = frame_id
                        tracked_objects[f'{track_id_best}']['iou'] = best_iou

                    else:
                        frame_boxes[i].append(track_id)
                        tracked_objects[f'{track_id}']  = {'bbox':boxA,'frame':frame_id,'memory':0,'iou':best_iou}
                        track_id += 1

                else:
                    frame_boxes[i].append(track_id_best)
                    tracked_objects[f'{track_id_best}']['bbox']= boxA
                    previous_f = tracked_objects[f'{track_id_best}']['frame']

                    # CHECK IF OBJECTS APPEAR CONSECUTIVE
                    if frame_id - previous_f == 1:
                        tracked_objects[f'{track_id_best}']['memory'] = tracked_objects[f'{track_id_best}']['memory'] + 1
                    tracked_objects[f'{track_id_best}']['frame'] = frame_id
                    tracked_objects[f'{track_id_best}']['iou'] = best_iou


    if frame_id == memory:
        track_memory(tracked_objects)
        memory = memory + frame_id 

    previous_tracked_objects = copy.deepcopy(tracked_objects)
    
            

In [9]:
frames = []
N = 150
path_results = './Results/task2_1'
path_results = os.path.join(current_path,path_results)
if not os.path.exists(path_results):
    os.makedirs(path_results)
for i in range(0,N):
    frame_visualization(video_capture,mask_rcnn_boxes,i,path_results,'mask_rcnn')


In [ ]:
previous_tracked_objects

In [ ]:
output = write_to_csv_file("task_2_1", mask_rcnn_boxes)

In [1]:
import pandas as pd

df_list = []
for frame_number, object_list in data.items():
    for object_data in object_list:
        frame = object_data[0]
        left = object_data[1]
        top = object_data[2]
        width = object_data[3] - object_data[1]
        height = object_data[4] - object_data[2]
        conf = object_data[5]
        if len(object_data) < 7:
            ob_id = -1
        else:
            ob_id = object_data[6]
        df_list.append(pd.DataFrame([frame, ob_id, left, top, width, height, conf, -1, -1, -1], index=[0]))
df = pd.concat(df_list, ignore_index=True)
df.to_csv("task_2_1.csv", header=["frame", "id", "left", "top", "width", "height", "confidence", "x", "y", "z"], index=False)

SyntaxError: incomplete input (3318536538.py, line 2)